ヒートマップの作成(heatdiff)
<方針><br>
・フェーズごとと全フェーズ合算の二種類で行う<br>
・各cell各フェーズにおいて、standard(標準化)のdiffを使う<br>
・標準化後の値が2.0以上のものを「活動状態」の判定基準とする<br>
・各cellのcolorが示す値は、そのエリアに存在した時に、「活動状態」だった確率<br>

最新バージョン(現在は不使用)保存用

In [ ]:
"""
    #Ca2+イメージング強度のslice間変化分(diff)が閾値(ts)より大きくなる頻度分布のヒートマップ:heatdiffprob(X:x,Y:y,heat:p(訪れた時にdiff?>ts))の作成
    #(現在は不使用)
    inputf='%s/python/%s_meandiffall.csv' %(mouse,mouse)          
    dfmeandiff_all=pd.read_csv(inputf,header=0)
    dfmap=pd.DataFrame(index=np.arange(195,-5,-200/20),columns=np.arange(6.25,256.25,250/20))
    dfmap_n=pd.DataFrame(index=np.arange(195,-5,-200/20),columns=np.arange(6.25,256.25,250/20))
    dfmap_d=pd.DataFrame(index=np.arange(195,-5,-200/20),columns=np.arange(6.25,256.25,250/20))

    for i in range(0,20):

        for j in range(0,20):

            n=dfmeandiff_all[(dfmeandiff_all['X']>float(dfmap.columns[i]-6.25) )& (dfmeandiff_all['X']<float(dfmap.columns[i]+6.25) )&( dfmeandiff_all['Y']>float(dfmap.index[j]-5) )& (dfmeandiff_all['Y']<float(dfmap.index[j]+5))].shape[0]

            dfmap_n.iloc[i,j]=n


    for ts in range(1,5):

        for cell in range(dfmeandiff_all.columns.get_loc(cell_f),dfmeandiff_all.columns.get_loc(cell_l)+1):

            for i in range(0,20):

                for j in range(0,20):

                    if dfmap_n.iloc[i,j]<1.0:


                        dfmap.iloc[i,j]=0

                        dfmap_d.iloc[i,j]=0

                    else:

                        d=dfmeandiff_all[(dfmeandiff_all['X']>float(dfmap.columns[i]-6.25)) & (dfmeandiff_all['X']<float(dfmap.columns[i]+6.25)) & (dfmeandiff_all['Y']>float(dfmap.index[j]-5)) & (dfmeandiff_all['Y']<float(dfmap.index[j]+5) )& (dfmeandiff_all[dfmeandiff_all.columns[cell]]>ts)].shape[0]


                        dfmap_d.iloc[i,j]=d

                        dfmap.iloc[i,j]=(d/dfmap_n.iloc[i,j])

            dfmap=dfmap.astype(float)
            #dfmap.columns.astype(float)
            #dfmap.index.astype(float)
            
            plt.figure()
            title_name='heatdiffprob_ts%s_%s' % (ts,dfmeandiff_all.columns[cell])
            file_name='%s/python/%s_fig/'%(mouse,mouse)+title_name+'.png'
            plt.title(title_name)
            plt.xlabel('x')
            plt.ylabel('y')
            sns.heatmap(dfmap)
            plt.savefig(file_name)

            dfmap_d=dfmap_d.astype(float)
            plt.figure()
            title_name='heatdiff_ts%s_%s' % (ts,dfmeandiff_all.columns[cell])
            file_name='%s/python/%s_fig/'%(mouse,mouse)+title_name+'.png'
            plt.title(title_name)
            plt.xlabel('x')
            plt.ylabel('y')
            sns.heatmap(dfmap_d)
            plt.savefig(file_name)
"""            

過去バージョン(現在は不使用) 保存用

In [1]:

"""
#for ph in range(0,len(phase_list)):

for ph in range(0,9):
   

    phase=phase_list[ph]

    #for index in range(0,len(Index)):

    inputXY='ID138464-%s_%s_XY.txt' %(mouse,phase)

    dfx = pd.read_table(inputXY,header=0)
    
    dfx['phase']=ph

    inputT='ID138464-%s_long_trace_%s.csv'%(mouse,phase)

    dfl=pd.read_csv(inputT,header=0)
    
    dfl_diff=dfl.diff()
    dfl_diff['Time1']=dfl['Time1']
    dfl_diff['Time2']=dfl['Time2']

    dfx_m=pd.DataFrame(columns=dfx.columns,index=np.arange(0,int(dfx.iloc[dfx.shape[0]-1].loc['Slice']-1)*10))

    for i in range(2,int(dfx.shape[0])):

        for j in range(0,10):

            for k in range(0,len(dfx_m.columns)):

                dfx_m.iloc[(i-2)*10+j].loc[dfx_m.columns[k]]=dfx.iloc[i].loc[dfx_m.columns[k]]

    df=pd.concat([dfx_m, dfl_diff], axis=1)
    df=df.dropna(how='any')

    Index=np.arange(int(df.iloc[0].loc['Slice']),int(df.iloc[df.shape[0]-1].loc['Slice']+1),1)

    df_mean=pd.DataFrame(index=Index,columns=[])

    for i in range(0,len(df.columns)):

        df_mean[df.columns[i]]=0

    for i_n in range(int(df.iloc[0].loc['Slice']),int(df.iloc[df.shape[0]-1].loc['Slice']+1)):

        i = i_n - int(df.iloc[0].loc['Slice'])

        df_mean.iloc[i].loc['Slice']=i_n

        dfs = df[df['Slice']==i_n]

        for k in range(0,df.columns.get_loc('Time2')+1):

            df_mean.iloc[i,k]=dfs.iloc[0,k]

        for j in range(dfs.columns.get_loc(' C000'),dfs.columns.get_loc(' C261')+1):

            df_mean.iloc[i,j]=dfs[dfs.columns[j]].mean()
            
    dfmean=df_mean.diff()
    dfmean.loc[:,:'Time2']=df_mean.loc[:,:'Time2']
    dfmean=dfmean.dropna(how='any')
    
    dfmean_std=pd.DataFrame(columns=dfmean.columns,index=dfmean.index)
    for k in range(dfmean_std.columns.get_loc('Slice'),dfmean_std.columns.get_loc(' C000')):
        dfmean_std[dfmean_std.columns[k]]=dfmean[dfmean_std.columns[k]]
    for i in range(dfmean_std.columns.get_loc(' C000'),dfmean_std.columns.get_loc(' C261')+1):
        for j in range(0,dfmean_std.shape[0]):
            dfmean_std.iloc[j,i]=((dfmean.iloc[j,i]-dfmean[dfmean_std.columns[i]].mean())/dfmean[dfmean_std.columns[i]].std())

    
    dfmean_all=pd.concat([dfmean_all,dfmean_std])

for j in range(dfmean_all.columns.get_loc(' C212'),dfmean_all.columns.get_loc(' C261')+1):
    
    dfmean_heat=pd.DataFrame()
    
    ts=4.0
    
    dfmean_heat=dfmean_all[dfmean_all[dfmean_all.columns[j]]>=ts]
    
    fig = plt.figure()
    ax = fig.add_subplot(111)

    H = ax.hist2d(dfmean_heat['X'],dfmean_heat['Y'], bins=20,range=[[0,250],[0,200]],normed=True,cmap=cm.jet)
    #ax.set_title('1st graph')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_xlim(0,250)
    ax.set_ylim(0,200)
    H[3].set_clim(0,0.003)
    fig.colorbar(H[3],ax=ax)
    
    #plt.show()


    plt.xlabel('x')
    plt.ylabel('y')             
    title_name='fig4/heatdiff_mouse%s_ts%s_%s' % (mouse,ts,dfmean_all.columns[j])
    plt.title(title_name)
    file_name=title_name+'.png'
    plt.savefig(file_name)
    plt.figure(figsize=(5.4, 3.6), dpi=80)
    plt.show()
    
"""

"\n#for ph in range(0,len(phase_list)):\n\nfor ph in range(0,9):\n   \n\n    phase=phase_list[ph]\n\n    #for index in range(0,len(Index)):\n\n    inputXY='ID138464-%s_%s_XY.txt' %(mouse,phase)\n\n    dfx = pd.read_table(inputXY,header=0)\n    \n    dfx['phase']=ph\n\n    inputT='ID138464-%s_long_trace_%s.csv'%(mouse,phase)\n\n    dfl=pd.read_csv(inputT,header=0)\n    \n    dfl_diff=dfl.diff()\n    dfl_diff['Time1']=dfl['Time1']\n    dfl_diff['Time2']=dfl['Time2']\n\n    dfx_m=pd.DataFrame(columns=dfx.columns,index=np.arange(0,int(dfx.iloc[dfx.shape[0]-1].loc['Slice']-1)*10))\n\n    for i in range(2,int(dfx.shape[0])):\n\n        for j in range(0,10):\n\n            for k in range(0,len(dfx_m.columns)):\n\n                dfx_m.iloc[(i-2)*10+j].loc[dfx_m.columns[k]]=dfx.iloc[i].loc[dfx_m.columns[k]]\n\n    df=pd.concat([dfx_m, dfl_diff], axis=1)\n    df=df.dropna(how='any')\n\n    Index=np.arange(int(df.iloc[0].loc['Slice']),int(df.iloc[df.shape[0]-1].loc['Slice']+1),1)\n\n    df_mean=

In [ ]:
max(3,5)